In [1]:
# default_exp core-sequence

# Actions in video

> API details.

In [2]:
#hide
from nbdev.showdoc import *

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import time
import datetime
from tqdm.notebook import tqdm

import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.autograd import Variable

from actions_in_videos.dataset_ucf101 import UCF101, SequenceDataset, DataLoader
from actions_in_videos.models import ResNet50Classifier, ConvLSTM
from actions_in_videos.dataset_ucf101 import SequenceBatchShower

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

In [4]:
print('torch.__version__', torch.__version__)

torch.__version__ 1.4.0


In [5]:
# save_dir = pathlib.Path('model-checkpoints')
# save_dir.mkdir(parents=True, exist_ok=True)

In [6]:
opt = {
    'channels':3,
    'img_dim':224,
    'sequence-length':16,
    'latent-dim': 512,
    'sample-interval': 1,
    'dataset-path': '../data/UCF101/',
    'batch-size': 32,
    'learning-rate': 0.0001,
    'num-epochs': 20,
    'checkpoint_interval': 10,
    'checkpoint-model': False
      }

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_shape = (opt['channels'], opt['img_dim'], opt['img_dim'])

# Normalization parameters for pre-trained PyTorch models
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

## Create the Training set and its associated Dataloader:

In [8]:
# Define training set
train_ds = SequenceDataset(
    dataset_path=opt['dataset-path'],
    sequence_length=opt['sequence-length'],
    sample_interval=opt['sample-interval'],
    training=True,
    transform=transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomCrop((224, 224)),
        transforms.Resize((opt['img_dim'], opt['img_dim']), Image.BICUBIC), 
        transforms.ToTensor(), 
        transforms.Normalize(mean, std),
    ])
)
train_dl = DataLoader(train_ds, batch_size=opt['batch-size'], shuffle=True, num_workers=4)


Number of training files: 9537


Let's make sure that we are loading the correct stuff:

In [9]:
# sb = SequenceBatchShower(train_dl, fig_size=5)
# sb.showBatch(3, scale=255, permuteToImg=(0,2,3,1))

# Define test set

In [10]:
test_ds = SequenceDataset(
    dataset_path=opt['dataset-path'],
    sequence_length=opt['sequence-length'],
    sample_interval=opt['sample-interval'],
    training=False,
    transform=transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((opt['img_dim'], opt['img_dim']), Image.BICUBIC), 
        transforms.ToTensor(), 
        transforms.Normalize(mean, std),
    ])
)


test_dl = DataLoader(test_ds, batch_size=opt['batch-size'], shuffle=False, num_workers=4)

Number of validation files: 3783


We can see what it contains. Note that in this case, there is no randomness so most likely all frames will be from the same video.

In [11]:
# sb = SequenceBatchShower(test_dl, fig_size=5)
# sb.showBatch(3, scale=255, permuteToImg=(0,2,3,1))

## Define the model

In [12]:
# Classification criterion
criterion = nn.CrossEntropyLoss().to(device)

# Define network
model = ConvLSTM(
    num_classes=train_ds.num_classes,
    latent_dim=opt['latent-dim'],
    lstm_layers=1,
    hidden_dim=1024,
    bidirectional=True,
    attention=True,
)
model = model.to(device)

Freeze the parameters of the parts we do not want to train and set to true those of the last few layers that we are interested in updating.

The dataset is not large enough to warrant training a full ResNet-50 model. 

To start, we will just fine-tune the output layer and the last BottleNeck block (or residual block). `model.layer4` contains three residual blocks implying model.layer4[2] is the last of these three residual blocks. Fine-tuning only the top layers also reduces the amount of GPU memory, meaning a higher batch size can be used and the model can be trained more quickly with less overfitting.

In [13]:
model

ConvLSTM(
  (encoder): Encoder(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)

In [16]:
for param in model.parameters():
    param.requires_grad_(False)
    
params = []
for param in model.lstm.parameters():
    param.requires_grad_(True)
    params.append(param)
for param in model.output_layers.parameters():
    param.requires_grad_(True)
    params.append(param)
for param in model.attention_layer.parameters():
    param.requires_grad_(True)
    params.append(param)



In [17]:
optimizer = torch.optim.Adam(params, lr=opt['learning-rate'])

In [18]:
if opt['checkpoint-model']:
    print(f"Loading checkpoint model: opt['checkpoint-model']")
    model.load_state_dict(torch.load(opt['checkpoint-model']))

## Training

We have now everything we need to build the training loop. This is done in the next two functions, the first one is to test the model on the validation set and the second one is the actual training loop.

In [19]:
def test_model(model,epoch):
    """ Evaluate the model on the test set """
    print("")
    model.eval()
    test_metrics = {"loss": [], "acc": []}
        
    for batch_i, (X, y) in enumerate(tqdm(test_dl, 'Test', leave=False)):                
        image_sequences = Variable(X.to(device), requires_grad=True).float()
        labels = Variable(y.to(device), requires_grad=False)
            
        labels = labels.squeeze()
        if len(labels.size()) == 0:
            labels = torch.tensor([labels]).to(device)
        
        with torch.no_grad():
            # Reset LSTM hidden state
            model.lstm.reset_hidden_state()
            # Get sequence predictions
            preds = model(image_sequences)
                    
        # Compute metrics
        acc = 100 * (preds.detach().argmax(1) == labels).cpu().numpy().mean()
        loss = criterion(preds, labels).item()
        
        # Keep track of loss and accuracy
        test_metrics["loss"].append(loss)
        test_metrics["acc"].append(acc)
        
        # Log test performance
        sys.stdout.write(
            "\r    Testing | Batch %d/%d | Loss: %f (%f), Acc: %.2f%% (%.2f%%) | "
            % (
                batch_i,
                len(test_dl),
                np.mean(test_metrics["loss"]),
                loss,
                np.mean(test_metrics["acc"]),
                acc,                
            )
        )    
    print("")

In [20]:
def train_model(model, opt, train_dl, test_dl):

    assert 'checkpoint-model' in opt.keys(), "Specify path to checkpoint model or set it to False"
    
    for epoch in tqdm(range(opt['num-epochs']), desc='Epoch #'):
        epoch_metrics = {"loss": [], "acc": []}
        prev_time = time.time()

        #print(f"--- Epoch {epoch} ---")
        for batch_i, (X, y) in enumerate(tqdm(train_dl, 'Train', leave=False)):
            model.train()

            if X.size(0) == 1:
                continue

            image_sequences = Variable(X.to(device), requires_grad=True).float()
            labels = Variable(y.to(device), requires_grad=False)
            
            labels = labels.squeeze()
            if len(labels.size()) == 0:
                labels = torch.tensor([labels]).to(device)

            optimizer.zero_grad()

            # Reset LSTM hidden state
            model.lstm.reset_hidden_state()

            # forward
            with torch.set_grad_enabled(True):
                # Get sequence predictions
                preds = model(image_sequences)

                # Compute metrics   
                loss = criterion(preds, labels)
                acc = 100 * (preds.detach().argmax(1) == labels).cpu().numpy().mean()

                loss.backward()
                optimizer.step()

            # Keep track of epoch metrics
            epoch_metrics["loss"].append(loss.item())
            epoch_metrics["acc"].append(acc)

            # Determine approximate time left
            batches_done = epoch * len(train_dl) + batch_i
            batches_left = opt['num-epochs'] * len(train_dl) - batches_done
            time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
            prev_time = time.time()

            # Print log
            sys.stdout.write(
                "\rEpoch %d/%d | Batch %d/%d | Loss: %f (%f), Acc: %.2f%% (%.2f%%) | ETA: %s"
                % (
                    epoch,
                    opt['num-epochs'],
                    batch_i,
                    len(train_dl),
                    np.mean(epoch_metrics["loss"]),
                    loss.item(),
                    np.mean(epoch_metrics["acc"]),                    
                    acc,
                    time_left,
                )
            )

#             Empty cache
            if torch.cuda.is_available():
                 torch.cuda.empty_cache()

        # Evaluate the model on the test set
        test_model(model, epoch)

        model.train()
        
        # Save model checkpoint
        if epoch % opt['checkpoint_interval'] == 0:
            os.makedirs('model-checkpoints', exist_ok=True)
            torch.save(model.state_dict(), f"model-checkpoints/{model.__class__.__name__}_{epoch}.pth")
            
    # Print log
    print("-------------------------------------------------------")
    sys.stdout.write(
        "\rEpoch %d/%d | Batch %d/%d | Loss: %f (%f), Acc: %.2f%% (%.2f%%) | ETA: %s"
        % (
            epoch+1,
            opt['num-epochs'],
            batch_i,
            len(train_dl),
            np.mean(epoch_metrics["loss"]),
            loss.item(),            
            np.mean(epoch_metrics["acc"]),
            acc,
            time_left,
        )
    )
    
    # save last model
    os.makedirs('model-checkpoints', exist_ok=True)
    random_int = int(np.random.uniform(0,1e14))
    torch.save(model.state_dict(), f"model-checkpoints/{model.__class__.__name__}_{epoch}_{random_int}.pth")
    print(f"Model saved as: model-checkpoints/{model.__class__.__name__}_{epoch}_{random_int}.pth")

In [ ]:
train_model(model, opt, train_dl, test_dl)

Epoch 0/20 | Batch 107/299 | Loss: 2.905571 (1.700179), Acc: 46.70% (84.38%) | ETA: 1:47:57.6161137

In [ ]:
# import random
# random_number = np.random.randint(0,16777215)
# hex_number = str(hex(random_number))
# print(hex_number)

In [ ]:
from nbdev.export import *
notebook2script()